# Movie Recommendation

In this project our goal is to build a movie recommendation system using collaborative filtering. Collaborative filtering is a learning technique used to make predictions (filtering) about the interests of a user by collecting preferences of taste information from many other users (collaboration). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue than that of a randomly chosen person.

For this task, the [Movielens dataset](https://grouplens.org/datasets/movielens/) was used. It contains 20 million ratings and 465000 tag applications applied to 27000 movies by 138000 users.

The Apache Spark framework (and its Machine Learning library) was used to process the data.

## Spark setup

As some functionalities of the Spark SQL component will be used, we need to initialize a `SparkSession` object.

The `SparkContext` object (required for other basic functionalities) is automatically created by PySpark and it is defined in the `sc` variable.

In [1]:
from pyspark.sql import SparkSession

In [2]:
ss = SparkSession.builder.appName("Movie Recommendation").getOrCreate()

## Data load

The MovieLens dataset is composed of three text files:

- `users.dat`: contains information about the genre, age, occupation and zipcode of each user.
- `movies.dat`: contains information about the title, year and genres of each movie.
- `ratings.dat`: contains the ratings that users gave to the movies they watched.

They can be opened as plain text RDDs.

In [3]:
import os

In [4]:
usersRDD = sc.textFile(os.path.join("ml-1m", "users.dat"))
usersRDD.take(5)

['1::F::1::10::48067',
 '2::M::56::16::70072',
 '3::M::25::15::55117',
 '4::M::45::7::02460',
 '5::M::25::20::55455']

In [5]:
moviesRDD = sc.textFile(os.path.join("ml-1m", "movies.dat"))
moviesRDD.take(5)

["1::Toy Story (1995)::Animation|Children's|Comedy",
 "2::Jumanji (1995)::Adventure|Children's|Fantasy",
 '3::Grumpier Old Men (1995)::Comedy|Romance',
 '4::Waiting to Exhale (1995)::Comedy|Drama',
 '5::Father of the Bride Part II (1995)::Comedy']

In [6]:
ratingsRDD = sc.textFile(os.path.join("ml-1m", "ratings.dat"))
ratingsRDD.take(5)

['1::1193::5::978300760',
 '1::661::3::978302109',
 '1::914::3::978301968',
 '1::3408::4::978300275',
 '1::2355::5::978824291']

## Data preparation

The plain text RDDs must be converted to structured data so we can use SQL queries and other high level operations to manipulate them. This is done for each RDD separately as each dataset has its own schema.

In [7]:
from pyspark.sql import Row
import datetime as dt
import re

### Users

According to the MovieLens dataset's documentation, each user record is composed of 5 fields:

- userID: User's unique identification.
- gender: User's gender ("F" for females, "M" for males).
- age: User's age range encoded as a number (see below).
- occupation: User's occupation encoded as a number (see below).
- zipcode: User's zipcode.

We usually need to encode categorical features as numbers (or sets of binary features) in order to use them in predictive models. Here, the `age` and `occupation` field values are already encoded; however, in a preliminary analysis it might be useful to know their actual values for the sake of interpretation. So this default encoding is undone and the codes are translated to the actual values they represent. Later, when needed, we may encode them again using Spark's `StringIndexer` class.

Thus, in the following cells the plain textual data for each user is split, the field values are computed, and the result is put into a structured `DataFrame`.

In [8]:
agesDict = {
    "1": "Under 18",
    "18": "18-24",
    "25": "25-34",
    "35": "35-44",
    "45": "45-49",
    "50": "50-55",
    "56": "Over 56",
}

In [9]:
occupationsDict = {
    "0": "other or unspecified",
    "1": "academic/educator",
    "2": "artist",
    "3": "clerical/admin",
    "4": "college/grad student",
    "5": "customer service",
    "6": "doctor/health care",
    "7": "executive/managerial",
    "8": "farmer",
    "9": "homemaker",
    "10": "K-12 student",
    "11": "lawyer",
    "12": "programmer",
    "13": "retired",
    "14": "sales/marketing",
    "15": "scientist",
    "16": "self-employed",
    "17": "technician/engineer",
    "18": "tradesman/craftsman",
    "19": "unemployed",
    "20": "writer",
}

In [10]:
usersRDD = usersRDD.map(lambda line: line.split("::")) \
                   .map(lambda t: Row(userID=int(t[0]),
                                      gender=t[1],
                                      age=agesDict[t[2]],
                                      occupation=occupationsDict[t[3]],
                                      zipcode=t[4]))
usersRDD.take(5)

[Row(age='Under 18', gender='F', occupation='K-12 student', userID=1, zipcode='48067'),
 Row(age='Over 56', gender='M', occupation='self-employed', userID=2, zipcode='70072'),
 Row(age='25-34', gender='M', occupation='scientist', userID=3, zipcode='55117'),
 Row(age='45-49', gender='M', occupation='executive/managerial', userID=4, zipcode='02460'),
 Row(age='25-34', gender='M', occupation='writer', userID=5, zipcode='55455')]

In [11]:
usersDF = ss.createDataFrame(usersRDD)
usersDF.show(5, truncate=False)

+--------+------+--------------------+------+-------+
|age     |gender|occupation          |userID|zipcode|
+--------+------+--------------------+------+-------+
|Under 18|F     |K-12 student        |1     |48067  |
|Over 56 |M     |self-employed       |2     |70072  |
|25-34   |M     |scientist           |3     |55117  |
|45-49   |M     |executive/managerial|4     |02460  |
|25-34   |M     |writer              |5     |55455  |
+--------+------+--------------------+------+-------+
only showing top 5 rows



### Movies

At first, each movie record is composed of 3 fields:

- movieID: Movie's unique identification.
- title: Movie's title and year (see below).
- genres: List of all genres the movie fits to.

The `title` field includes the movie's year because there are movies with same name made (or remade) in different epochs, and the year information would be the only way to distinct them. However, as we have a `movieID` field, this is not really necessary. So we can extract the year from the title and make another field.

Thus, in the following cells the plain textual data for each movie is split, the field values are computed, and the result is put into a structured `DataFrame`.

In [12]:
moviesRDD = moviesRDD.map(lambda line: line.split("::")) \
                     .map(lambda t: (t[0], re.search(r"(.+)\s\((\d{4})\)", t[1]), t[2])) \
                     .map(lambda t: Row(movieID=int(t[0]),
                                        title=t[1].group(1),
                                        year=int(t[1].group(2)),
                                        genres=t[2]))
moviesRDD.take(5)

[Row(genres="Animation|Children's|Comedy", movieID=1, title='Toy Story', year=1995),
 Row(genres="Adventure|Children's|Fantasy", movieID=2, title='Jumanji', year=1995),
 Row(genres='Comedy|Romance', movieID=3, title='Grumpier Old Men', year=1995),
 Row(genres='Comedy|Drama', movieID=4, title='Waiting to Exhale', year=1995),
 Row(genres='Comedy', movieID=5, title='Father of the Bride Part II', year=1995)]

In [13]:
moviesDF = ss.createDataFrame(moviesRDD)
moviesDF.show(5, truncate=False)

+----------------------------+-------+---------------------------+----+
|genres                      |movieID|title                      |year|
+----------------------------+-------+---------------------------+----+
|Animation|Children's|Comedy |1      |Toy Story                  |1995|
|Adventure|Children's|Fantasy|2      |Jumanji                    |1995|
|Comedy|Romance              |3      |Grumpier Old Men           |1995|
|Comedy|Drama                |4      |Waiting to Exhale          |1995|
|Comedy                      |5      |Father of the Bride Part II|1995|
+----------------------------+-------+---------------------------+----+
only showing top 5 rows



### Ratings

Each rating data is composed of 4 fields:

- userID: ID of the user who gave the rating.
- movieID: ID of the movie which was rated.
- rating: A numerical value ranging from 1 (min) to 5 (max).
- timestamp: A number that encodes the date and time the rating was given at.

The `timestamp` can be easily converted to a `datetime` object, which is much easier to interpret.

Thus, in the following cells the plain textual data for each rating is split, the field values are computed, and the result is put into a structured DataFrame.

In [14]:
ratingsRDD = ratingsRDD.map(lambda line: line.split("::")) \
                       .map(lambda t: Row(userID=int(t[0]),
                                          movieID=int(t[1]),
                                          rating=float(t[2]),
                                          timestamp=dt.datetime.fromtimestamp(int(t[3]))))
ratingsRDD.take(5)

[Row(movieID=1193, rating=5.0, timestamp=datetime.datetime(2000, 12, 31, 20, 12, 40), userID=1),
 Row(movieID=661, rating=3.0, timestamp=datetime.datetime(2000, 12, 31, 20, 35, 9), userID=1),
 Row(movieID=914, rating=3.0, timestamp=datetime.datetime(2000, 12, 31, 20, 32, 48), userID=1),
 Row(movieID=3408, rating=4.0, timestamp=datetime.datetime(2000, 12, 31, 20, 4, 35), userID=1),
 Row(movieID=2355, rating=5.0, timestamp=datetime.datetime(2001, 1, 6, 21, 38, 11), userID=1)]

In [15]:
ratingsDF = ss.createDataFrame(ratingsRDD)
ratingsDF.show(5, truncate=False)

+-------+------+---------------------+------+
|movieID|rating|timestamp            |userID|
+-------+------+---------------------+------+
|1193   |5.0   |2000-12-31 20:12:40.0|1     |
|661    |3.0   |2000-12-31 20:35:09.0|1     |
|914    |3.0   |2000-12-31 20:32:48.0|1     |
|3408   |4.0   |2000-12-31 20:04:35.0|1     |
|2355   |5.0   |2001-01-06 21:38:11.0|1     |
+-------+------+---------------------+------+
only showing top 5 rows



## Exploratory data analysis

Here we'll just do some exploration on the structured datasets in order to better understand all the data we have.

### Users

In [16]:
usersDF.createOrReplaceTempView("users")

In [17]:
# Get the gender distribution.
ss.sql("""SELECT gender, COUNT(gender) as frequency
          FROM users
          GROUP BY gender
          ORDER BY frequency DESC""").show()

+------+---------+
|gender|frequency|
+------+---------+
|     M|     4331|
|     F|     1709|
+------+---------+



In [18]:
# Get the age distribution.
ss.sql("""SELECT age, COUNT(age) as frequency
          FROM users
          GROUP BY age
          ORDER BY frequency DESC""").show()

+--------+---------+
|     age|frequency|
+--------+---------+
|   25-34|     2096|
|   35-44|     1193|
|   18-24|     1103|
|   45-49|      550|
|   50-55|      496|
| Over 56|      380|
|Under 18|      222|
+--------+---------+



In [19]:
# Get the occupation distribution.
ss.sql("""SELECT occupation, COUNT(occupation) as frequency
          FROM users
          GROUP BY occupation
          ORDER BY frequency DESC""").show(21)

+--------------------+---------+
|          occupation|frequency|
+--------------------+---------+
|college/grad student|      759|
|other or unspecified|      711|
|executive/managerial|      679|
|   academic/educator|      528|
| technician/engineer|      502|
|          programmer|      388|
|     sales/marketing|      302|
|              writer|      281|
|              artist|      267|
|       self-employed|      241|
|  doctor/health care|      236|
|        K-12 student|      195|
|      clerical/admin|      173|
|           scientist|      144|
|             retired|      142|
|              lawyer|      129|
|    customer service|      112|
|           homemaker|       92|
|          unemployed|       72|
| tradesman/craftsman|       70|
|              farmer|       17|
+--------------------+---------+



In [20]:
ss.catalog.dropTempView("users")

### Movies

In [21]:
moviesDF.createOrReplaceTempView("movies")

In [22]:
# # Get the year distribution.
# ss.sql("""SELECT year, COUNT(year) as frequency
#           FROM movies
#           GROUP BY year
#           ORDER BY frequency DESC""").show()

In [23]:
ss.catalog.dropTempView("movies")

### Ratings

In [24]:
ratingsDF.createOrReplaceTempView("ratings")

In [25]:
# Get the rating distribution.
ss.sql("""SELECT rating, COUNT(rating) as frequency
          FROM ratings
          GROUP BY rating
          ORDER BY frequency DESC""").show()

+------+---------+
|rating|frequency|
+------+---------+
|   4.0|   348971|
|   3.0|   261197|
|   5.0|   226310|
|   2.0|   107557|
|   1.0|    56174|
+------+---------+



In [26]:
# Get the most active users and their average ratings.
ss.sql("""SELECT userID, COUNT(userID) as numRatings, AVG(rating) as avgRating
          FROM ratings
          GROUP BY userID
          ORDER BY numRatings DESC""").show()

+------+----------+------------------+
|userID|numRatings|         avgRating|
+------+----------+------------------+
|  4169|      2314|3.5518582541054453|
|  1680|      1850|3.5556756756756758|
|  4277|      1743| 4.134825014343087|
|  1941|      1595|3.0545454545454547|
|  1181|      1521| 2.815910585141354|
|   889|      1518|2.8405797101449277|
|  3618|      1344|3.0081845238095237|
|  2063|      1323| 2.945578231292517|
|  1150|      1302|2.5906298003072195|
|  1015|      1286|3.7262830482115086|
|  5795|      1277|3.0563821456538762|
|  4344|      1271| 3.331235247836349|
|  1980|      1260|3.4825396825396826|
|  2909|      1258|3.8227344992050876|
|  1449|      1243|2.8061142397425582|
|  4510|      1240|2.8379032258064516|
|   424|      1226| 3.735725938009788|
|  4227|      1222|2.7013093289689034|
|  5831|      1220|3.6778688524590164|
|  3391|      1216|3.7203947368421053|
+------+----------+------------------+
only showing top 20 rows



In [27]:
# Get the most rated movies and their average ratings.
ss.sql("""SELECT movieID, COUNT(movieID) as numRatings, AVG(rating) as avgRating
          FROM ratings
          GROUP BY movieID
          ORDER BY numRatings DESC""").show()

+-------+----------+------------------+
|movieID|numRatings|         avgRating|
+-------+----------+------------------+
|   2858|      3428|4.3173862310385065|
|    260|      2991| 4.453694416583082|
|   1196|      2990| 4.292976588628763|
|   1210|      2883| 4.022892819979188|
|    480|      2672|3.7638473053892216|
|   2028|      2653| 4.337353938937053|
|    589|      2649| 4.058512646281616|
|   2571|      2590| 4.315830115830116|
|   1270|      2583|3.9903213317847466|
|    593|      2578|4.3518231186966645|
|   1580|      2538| 3.739952718676123|
|   1198|      2514| 4.477724741447892|
|    608|      2513| 4.254675686430561|
|   2762|      2459| 4.406262708418057|
|    110|      2443| 4.234957020057307|
|   2396|      2369| 4.127479949345715|
|   1197|      2318|4.3037100949094045|
|    527|      2304| 4.510416666666667|
|   1617|      2288| 4.219405594405594|
|   1265|      2278| 3.953028972783143|
+-------+----------+------------------+
only showing top 20 rows



In [28]:
ss.catalog.dropTempView("ratings")

## Preprocessing

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import IndexToString

## Model training

In [ ]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

## Results